In [1]:
import pandas as pd
from collections import Counter, defaultdict
import random
import math
import numpy as np

In [4]:
import json
from collections import defaultdict, Counter
with open("tokenized.json", "r", encoding="utf-8") as f:
    data = json.load(f)
all_sentences = [sentence for sentence in data["sentences"]]
all_tokens = [token for sentence in data["tokens"] for token in sentence]
print(f"Successfully loaded {len(all_sentences)} sentences.")
print(f"Total tokens for training: {len(all_tokens)}")

Successfully loaded 1557 sentences.
Total tokens for training: 21856


In [5]:
random.seed(42) 
test_sentences = random.sample(all_sentences, 1000)
print(f"Randomly selected {len(test_sentences)} sentences for testing.\n")

Randomly selected 1000 sentences for testing.



In [6]:
unigram_counts = Counter(all_tokens)
bigram_counts = Counter(zip(all_tokens, all_tokens[1:]))

V = len(unigram_counts)
print(f"Vocabulary Size (V): {V}")

#Pre-compute T(w)->the number of unique token types that follow each word w
#Token Type Smoothing
following_types = defaultdict(set)
for w1, w2 in bigram_counts:
    following_types[w1].add(w2)
T = {word: len(types) for word, types in following_types.items()}
print("Pre-computation complete.\n")

Vocabulary Size (V): 8264
Pre-computation complete.



In [7]:
def calculate_add_one_prob(sentence, unigram_counts, bigram_counts, V):
    #Add-One (Laplace) smoothing
    log_prob = 0.0
    for i in range(len(sentence) - 1):
        w1, w2 = sentence[i], sentence[i+1]
        bigram = (w1, w2)
        
        # Formula: P(w2 | w1) = (count(w1, w2) + 1) / (count(w1) + V)
        numerator = bigram_counts.get(bigram, 0) + 1
        denominator = unigram_counts.get(w1, 0) + V
        
        log_prob += np.log(numerator / denominator)
        
    return log_prob

In [8]:
def calculate_add_k_prob(sentence, unigram_counts, bigram_counts, V, k):
    #Add-K smoothing
    log_prob = 0.0
    for i in range(len(sentence) - 1):
        w1, w2 = sentence[i], sentence[i+1]
        bigram = (w1, w2)
        
        # Formula: P(w2 | w1) = (count(w1, w2) + k) / (count(w1) + k*V)
        numerator = bigram_counts.get(bigram, 0) + k
        denominator = unigram_counts.get(w1, 0) + k * V
        
        log_prob += np.log(numerator / denominator)
        
    return log_prob

In [9]:
def calculate_token_type_prob(sentence, unigram_counts, bigram_counts, T):
    #Token Type" smoothing.
    #P(w2 | w1) = (count(w1, w2) + 1) / (count(w1) + T(w1))
    #T(w1)->number of unique word types that follow w1.

    log_prob = 0.0
    for i in range(len(sentence) - 1):
        w1, w2 = sentence[i], sentence[i+1]
        bigram = (w1, w2)
        
        # Get count of unique followers for w1, default to V if w1 is unknown
        num_following_types = T.get(w1, V)
        
        numerator = bigram_counts.get(bigram, 0) + 1
        denominator = unigram_counts.get(w1, 0) + num_following_types
        
        # Avoid division by zero if a token somehow has 0 count and 0 followers
        if denominator == 0:
            continue

        log_prob += np.log(numerator / denominator)
        
    return log_prob

In [10]:
k_value = 0.1 

results = []

for sentence in test_sentences:
    #sentence to short for bi-gram
    if len(sentence) < 2:
        continue
    
    prob_add_one = calculate_add_one_prob(sentence, unigram_counts, bigram_counts, V)
    prob_add_k = calculate_add_k_prob(sentence, unigram_counts, bigram_counts, V, k_value)
    prob_token_type = calculate_token_type_prob(sentence, unigram_counts, bigram_counts, T)
    
    results.append({
        "sentence": " ".join(sentence),
        "add_one_log_prob": prob_add_one,
        "add_k_log_prob": prob_add_k,
        "token_type_log_prob": prob_token_type
    })

In [13]:
print(f"(A higher log probability, closer to 0, indicates a better fit by the model)\n")

#for first 5 sentences
for i in range(5):
    res = results[i]
    print(f"Sentence {i+1}: \"{res['sentence']}\"")
    print(f"Add-One Smoothing Log Prob:      {res['add_one_log_prob']:.4f}")
    print(f"Add-K (k={k_value}) Smoothing Log Prob: {res['add_k_log_prob']:.4f}")
    print(f"Token Type Smoothing Log Prob:   {res['token_type_log_prob']:.4f}\n")

(A higher log probability, closer to 0, indicates a better fit by the model)

Sentence 1: "સ ં ય ુ ક ્ ત   પ ્ ર સ ્ ત ા વ મ ા ં   મ ા ં ગ   ક ર વ ા મ ા ં   આ વ ી   હ ત ી   ક ે ,   વ ડ ા પ ્ ર ધ ા ન   ન ર ે ન ્ દ ્ ર   મ ો દ ી ન ી   સ ર ક ા ર મ ા ં   લ ો ક ત ા ં ત ્ ર િ ક   અ સ ં મ ત િ ન ે   સ ત ્ ત ા ન ા   જ ો ર ે   દ બ ા વ વ ા મ ા ં   આ વ ી   ર હ ી   છ ે"
Add-One Smoothing Log Prob:      -1199.7360
Add-K (k=0.1) Smoothing Log Prob: -1200.7257
Token Type Smoothing Log Prob:   -928.5598

Sentence 2: "જ ો   પ ્ ર જ ા   આ ક ્ ર મ ક   બ ન શ ે   અ ન ે   ત ા ળ બ ં ધ ી   સ ુ ધ ી ન ી   ત ૈ ય ા ર ી   દ ર ્ શ ા વ શ ે   ત ો   પ ્ ર જ ા   મ ા ટ ે   આ   ન ા ણ ા   ઉ પ ય ો ગ   ક ર વ ા ન ી   ત સ ્ દ ી   ક દ ા ચ   ન ા છ ુ ટ ક ે   આ   બ ં દ ર ન ે   લ ે વ ા ન ી   ફ ર જ   પ ડ ી   શ ક ે   છ ે"
Add-One Smoothing Log Prob:      -1244.9214
Add-K (k=0.1) Smoothing Log Prob: -1246.6235
Token Type Smoothing Log Prob:   -982.1496

Sentence 3: "ક ો ઈ   ક ં પ ન ી   ત ે ઓ   સ ં બ ં ધ િ ત   છ ે ,   સ ા ં જ ે   ચ ા  

In [12]:
print(all_tokens[:50])


['આ', 'વીડિયો', 'જુઓ', 'ઊંઝા', 'માર્કેટયાર્ડ', 'આજથી', 'જુલાઈ', 'સુધી', 'બંધ', 'મિથેનોલ', 'આવ્યો', 'ક્યાંથી', 'આખરે', 'ત્રણ', 'રાજ્યોમાં', 'મળેલ', 'હાર', 'પર', 'કોંગ્રેસ', 'અધ્યક્ષ', 'રાહુલ', 'ગાંધી', 'દ્વારા', 'પ્રથમ', 'પ્રતિક્રિયા', 'આપવામાં', 'આવી', 'છે', 'તેમણે', 'કહ્યું', 'કે', 'ત્રિપુરા', 'નાગાલેન્ડ', 'અને', 'મેઘાલયમાં', 'લોકોના', 'જનાદેશનો', 'સ્વાગત', 'કરીએ', 'છે', 'અને', 'આ', 'ક્ષેત્રના', 'લોકોનો', 'વિશ્વાસ', 'ફરીથી', 'જીતીવા', 'માટે', 'પ્રતિબદ્ધ', 'છીએ']
